### Train ETH data to CNN generative network

In [4]:
import torch
import numpy as np
from ReadData import ReadETHFolder, ReadETHFile
foldername="./ETH_Data/v/"
currentname = "./ETH_Data/"+"currents_3787.h5"
file_num = 10
data_shape = (16,16,16,3)
Bfield = torch.tensor(ReadETHFolder(foldername,file_num, data_shape)).permute(0,4,1,2,3)
current = torch.tensor(ReadETHFile(currentname))


In [5]:
print(Bfield.shape)
print(current.shape)

torch.Size([10, 3, 16, 16, 16])
torch.Size([3787, 8])
